In [8]:
import os
import numpy as np
from PIL import Image

# Input folders
input_folder = "train"
image_folder = os.path.join(input_folder, "images")
label_folder = os.path.join(input_folder, "labels")

# Output root prefix and levels
output_root_prefix = "train_permutated"
patch_levels = [5, 10, 15]  # patch sizes for three levels

# Create output directories for each patch level
for level in patch_levels:
    out_img = os.path.join(f"{output_root_prefix}_{level}", "images")
    out_lbl = os.path.join(f"{output_root_prefix}_{level}", "labels")
    os.makedirs(out_img, exist_ok=True)
    os.makedirs(out_lbl, exist_ok=True)

# --- Noise functions ---
def add_salt_noise(image, amount=0.02):
    noisy = image.copy()
    num_salt = int(amount * image.size / image.shape[2])
    coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = [255, 255, 255]
    return noisy

def add_pepper_noise(image, amount=0.02):
    noisy = image.copy()
    num_pepper = int(amount * image.size / image.shape[2])
    coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = [0, 0, 0]
    return noisy

def add_gaussian_noise(image, mean=0, sigma=20):
    gauss = np.random.normal(mean, sigma, image.shape).astype(np.int16)
    noisy = np.clip(image.astype(np.int16) + gauss, 0, 255).astype(np.uint8)
    return noisy

# --- Patch-level permutation ---
def perturbate_objects(image_path, label_path, noise=None, patch_size=10):
    """
    Keep class 0 and 1 bounding boxes intact.
    Permute remaining regions in non-overlapping patch_size x patch_size blocks.
    Handles edge patches properly.
    """
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img)
    img_h, img_w, _ = img_array.shape

    with open(label_path, "r") as f:
        lines = f.readlines()

    modified_array = img_array.copy()

    # Mask for class 0 and 1 bounding boxes
    mask = np.zeros((img_h, img_w), dtype=bool)
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        cls, x, y, w, h = parts
        if cls not in ["0", "1"]:
            continue
        x, y, w, h = map(float, [x, y, w, h])
        left = int(x * img_w - (w * img_w) / 2)
        upper = int(y * img_h - (h * img_h) / 2)
        right = int(x * img_w + (w * img_w) / 2)
        lower = int(y * img_h + (h * img_h) / 2)
        left, upper = max(0, left), max(0, upper)
        right, lower = min(img_w, right), min(img_h, lower)
        mask[upper:lower, left:right] = True  # fixed area mask

    # Collect all patch coordinates for background (outside mask)
    patches = []
    coords = []
    for y in range(0, img_h, patch_size):
        for x in range(0, img_w, patch_size):
            y_end = min(y + patch_size, img_h)
            x_end = min(x + patch_size, img_w)
            patch_mask = mask[y:y_end, x:x_end]
            if not patch_mask.any():  # only permute if patch doesn't overlap any fixed box
                patches.append(modified_array[y:y_end, x:x_end].copy())
                coords.append((y, y_end, x, x_end))

    # Shuffle patches
    np.random.shuffle(patches)

    # Put shuffled patches back (handle variable shape at edges)
    for (y, y_end, x, x_end), patch in zip(coords, patches):
        h_patch, w_patch, _ = patch.shape
        h_target = y_end - y
        w_target = x_end - x
        if h_patch != h_target or w_patch != w_target:
            # Resize patch to match target region
            patch_resized = np.array(Image.fromarray(patch).resize((w_target, h_target), Image.BICUBIC))
            modified_array[y:y_end, x:x_end] = patch_resized
        else:
            modified_array[y:y_end, x:x_end] = patch

    # Optional noise
    if noise == "salt":
        modified_array = add_salt_noise(modified_array, amount=0.03)
    elif noise == "pepper":
        modified_array = add_pepper_noise(modified_array, amount=0.03)
    elif noise == "gauss":
        modified_array = add_gaussian_noise(modified_array, sigma=25)

    return Image.fromarray(modified_array)


# --- Dataset processing for multiple patch levels ---
def process_dataset():
    for img_name in os.listdir(image_folder):
        if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        base_name, ext = os.path.splitext(img_name)
        img_path = os.path.join(image_folder, img_name)
        label_path = os.path.join(label_folder, base_name + ".txt")

        if not os.path.exists(label_path):
            continue

        # Read labels
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Process for each patch level
        for level in patch_levels:
            out_img_folder = os.path.join(f"{output_root_prefix}_{level}", "images")
            out_lbl_folder = os.path.join(f"{output_root_prefix}_{level}", "labels")

            # Save original image and labels (duplicates in each level folder)
            Image.open(img_path).save(os.path.join(out_img_folder, img_name))
            with open(os.path.join(out_lbl_folder, base_name + ".txt"), "w") as f:
                f.writelines(lines)

            # Permuted version
            perm_img = perturbate_objects(img_path, label_path, patch_size=level)
            perm_img.save(os.path.join(out_img_folder, base_name + "_perm" + ext))
            with open(os.path.join(out_lbl_folder, base_name + "_perm.txt"), "w") as f:
                f.writelines(lines)

            # Permuted + noise versions
            for noise in ["salt", "pepper", "gauss"]:
                noisy_img = perturbate_objects(img_path, label_path, noise=noise, patch_size=level)
                noisy_img.save(os.path.join(out_img_folder, base_name + f"_perm_{noise}" + ext))
                with open(os.path.join(out_lbl_folder, base_name + f"_perm_{noise}.txt"), "w") as f:
                    f.writelines(lines)

    print("✅ Finished generating patch-level permuted datasets for levels:", patch_levels)

if __name__ == "__main__":
    process_dataset()


✅ Finished generating patch-level permuted datasets for levels: [5, 10, 15]


In [10]:
import os
import numpy as np
from PIL import Image

# Input folders
input_folder = "train"
image_folder = os.path.join(input_folder, "images")
label_folder = os.path.join(input_folder, "labels")

# Output root prefix and levels
output_root_prefix = "train_permutated"
patch_levels = [20,30,40,50,60,70,80,90]  # patch sizes for three levels

# Create output directories for each patch level
for level in patch_levels:
    out_img = os.path.join(f"{output_root_prefix}_{level}", "images")
    out_lbl = os.path.join(f"{output_root_prefix}_{level}", "labels")
    os.makedirs(out_img, exist_ok=True)
    os.makedirs(out_lbl, exist_ok=True)

# --- Noise functions ---
def add_salt_noise(image, amount=0.02):
    noisy = image.copy()
    num_salt = int(amount * image.size / image.shape[2])
    coords = [np.random.randint(0, i - 1, num_salt) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = [255, 255, 255]
    return noisy

def add_pepper_noise(image, amount=0.02):
    noisy = image.copy()
    num_pepper = int(amount * image.size / image.shape[2])
    coords = [np.random.randint(0, i - 1, num_pepper) for i in image.shape[:2]]
    noisy[coords[0], coords[1]] = [0, 0, 0]
    return noisy

def add_gaussian_noise(image, mean=0, sigma=20):
    gauss = np.random.normal(mean, sigma, image.shape).astype(np.int16)
    noisy = np.clip(image.astype(np.int16) + gauss, 0, 255).astype(np.uint8)
    return noisy

# --- Patch-level permutation ---
def perturbate_objects(image_path, label_path, noise=None, patch_size=10):
    """
    Keep class 0 and 1 bounding boxes intact.
    Permute remaining regions in non-overlapping patch_size x patch_size blocks.
    Handles edge patches properly.
    """
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img)
    img_h, img_w, _ = img_array.shape

    with open(label_path, "r") as f:
        lines = f.readlines()

    modified_array = img_array.copy()

    # Mask for class 0 and 1 bounding boxes
    mask = np.zeros((img_h, img_w), dtype=bool)
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue
        cls, x, y, w, h = parts
        if cls not in ["0", "1"]:
            continue
        x, y, w, h = map(float, [x, y, w, h])
        left = int(x * img_w - (w * img_w) / 2)
        upper = int(y * img_h - (h * img_h) / 2)
        right = int(x * img_w + (w * img_w) / 2)
        lower = int(y * img_h + (h * img_h) / 2)
        left, upper = max(0, left), max(0, upper)
        right, lower = min(img_w, right), min(img_h, lower)
        mask[upper:lower, left:right] = True  # fixed area mask

    # Collect all patch coordinates for background (outside mask)
    patches = []
    coords = []
    for y in range(0, img_h, patch_size):
        for x in range(0, img_w, patch_size):
            y_end = min(y + patch_size, img_h)
            x_end = min(x + patch_size, img_w)
            patch_mask = mask[y:y_end, x:x_end]
            if not patch_mask.any():  # only permute if patch doesn't overlap any fixed box
                patches.append(modified_array[y:y_end, x:x_end].copy())
                coords.append((y, y_end, x, x_end))

    # Shuffle patches
    np.random.shuffle(patches)

    # Put shuffled patches back (handle variable shape at edges)
    for (y, y_end, x, x_end), patch in zip(coords, patches):
        h_patch, w_patch, _ = patch.shape
        h_target = y_end - y
        w_target = x_end - x
        if h_patch != h_target or w_patch != w_target:
            # Resize patch to match target region
            patch_resized = np.array(Image.fromarray(patch).resize((w_target, h_target), Image.BICUBIC))
            modified_array[y:y_end, x:x_end] = patch_resized
        else:
            modified_array[y:y_end, x:x_end] = patch

    # Optional noise
    if noise == "salt":
        modified_array = add_salt_noise(modified_array, amount=0.03)
    elif noise == "pepper":
        modified_array = add_pepper_noise(modified_array, amount=0.03)
    elif noise == "gauss":
        modified_array = add_gaussian_noise(modified_array, sigma=25)

    return Image.fromarray(modified_array)


# --- Dataset processing for multiple patch levels ---
def process_dataset():
    for img_name in os.listdir(image_folder):
        if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        base_name, ext = os.path.splitext(img_name)
        img_path = os.path.join(image_folder, img_name)
        label_path = os.path.join(label_folder, base_name + ".txt")

        if not os.path.exists(label_path):
            continue

        # Read labels
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Process for each patch level
        for level in patch_levels:
            out_img_folder = os.path.join(f"{output_root_prefix}_{level}", "images")
            out_lbl_folder = os.path.join(f"{output_root_prefix}_{level}", "labels")

            # Save original image and labels (duplicates in each level folder)
            Image.open(img_path).save(os.path.join(out_img_folder, img_name))
            with open(os.path.join(out_lbl_folder, base_name + ".txt"), "w") as f:
                f.writelines(lines)

            # Permuted version
            perm_img = perturbate_objects(img_path, label_path, patch_size=level)
            perm_img.save(os.path.join(out_img_folder, base_name + "_perm" + ext))
            with open(os.path.join(out_lbl_folder, base_name + "_perm.txt"), "w") as f:
                f.writelines(lines)

            # Permuted + noise versions
            for noise in ["salt", "pepper", "gauss"]:
                noisy_img = perturbate_objects(img_path, label_path, noise=noise, patch_size=level)
                noisy_img.save(os.path.join(out_img_folder, base_name + f"_perm_{noise}" + ext))
                with open(os.path.join(out_lbl_folder, base_name + f"_perm_{noise}.txt"), "w") as f:
                    f.writelines(lines)

    print("✅ Finished generating patch-level permuted datasets for levels:", patch_levels)

if __name__ == "__main__":
    process_dataset()


✅ Finished generating patch-level permuted datasets for levels: [20, 30, 40, 50, 60, 70, 80, 90]
